# Load previously scrubbed FCC data and plot

Kerry N. Wood

kerry.wood@jhuapl.edu

May 6, 2022

-  antimereidian hack: https://towardsdatascience.com/around-the-world-in-80-lines-crossing-the-antimeridian-with-python-and-shapely-c87c9b6e1513

In [ ]:
import json
import pandas as pd
import geopandas as gpd
import glob
import os

from shapely.geometry import shape

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# world.plot( figsize=(15,5) )

----

Read contours and annotate with GeoPandas geometries

----

In [ ]:
# raw contour data
with open('./data/contours_20220505.json') as F: t_contours = json.load(F)

In [ ]:
# flatten the data
flat = []
for V in t_contours.values(): flat.extend( V )
t_contours_df = pd.DataFrame( flat )

# make geometries, and set the beamId as the index
shapes = t_contours_df['geographyJson'].apply( lambda X: shape(X) )
t_contours_df[ 'geometry' ] = gpd.GeoSeries( shapes )
t_contours_df[ 'myidx' ] = t_contours_df[ 'idBeam'].copy()
t_contours_df.set_index( 'myidx', inplace=True )

------

All other transponder and frequency info

------

In [ ]:
# UDL channels, beam, RF data
t_udl = pd.read_json('./derived_data/udl_merged_no_contours.json')

-----

Pull out "real" entries

------

In [ ]:
real = t_udl[ t_udl['dataMode_channels'].str.contains('REAL')]
real.columns

# "REAL" data

### NORAD

In [ ]:
sc = pd.read_csv('./data/satcat.csv')
lookup = { X['NORAD_CAT_ID'] : X for X in sc.to_dict('records')}

def getName( X ):
    try: return lookup.get(X)['OBJECT_NAME']
    except: return "UNK"
real['OBJECT_NAME'] = real['NORAD'].apply( getName )
real['OBJECT_NAME'].unique()

In [ ]:
# TEST
# ses = real[ real['OBJECT_NAME'].str.contains('SES-1')]
chosensat = real[ real['OBJECT_NAME'].str.contains('SES-3')].copy()
# chosensat = real[ real['OBJECT_NAME'].str.contains('AFRISTAR')].copy()
chosensat.reset_index( inplace=True )
chosensat

In [ ]:
import matplotlib.pyplot as plt

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
plt.close('all')

# clear out all previous plots
files = sorted( glob.glob('./beam_plots/*.png' ) )
for F in files:
    print('Removing {}'.format(F),end='\r')
    os.remove( F )
print()

for i,R in chosensat.iterrows():
    print(i,end='\r')
    beam = R['idBeam']   # get the beam identifier
    thisbeam = gpd.GeoDataFrame( t_contours_df.loc[ beam ] ) # find that in the beam frame and turn into GeoPandas
    if thisbeam.shape[0] == 0: continue
#     print(thisbeam)
#     print(beam.columns)
    
    plt.figure()
    fig, ax = plt.subplots(figsize=(15, 15))
    world.plot( ax=ax, figsize=(15,5) )
    try: thisbeam.plot( ax=ax, column='gain',alpha=0.5 )
    except : 
        print(thisbeam)
        continue
    titlestr = '{:05d} -- mode: {} -- center freq: {:10.2f} -- band: {}'.format(
        int(R['NORAD']), R['mode'], R['centerFreq'], R['band'])
    ax.set_title(titlestr)
    plt.savefig('./beam_plots/{}.png'.format(i))
    plt.close()
#     if i > 30 : break

# SES-3: freaking weird

- SES-3 has an undisclosed Ka pyaload that shows up in the FCC database, but not tied to an uplink, 
- ex: `['37748-8466-KA020', 'TX', 'Ku', 17490.65, 'AT020'],`

https://spacenews.com/32392ses-3-sat-took-mystery-tour-with-secret-ka-band-payload/


In [ ]:
weirdos = []
for i,G in real.groupby('idTransponder'):
    if len(G['mode'].unique()) != 2: 
        weirdos.append( G )
weirdos = pd.concat(weirdos)

In [ ]:
# weirdos
# real[ real['OBJECT_NAME'].str.contains('INTELSAT 702')][['idTransponder','mode','band','name_channels']]
# weirdos.iloc[0]

In [ ]:
real[ real['OBJECT_NAME'].str.contains('SES-3')][['idTransponder','mode','band','centerFreq','name_channels']].values

In [ ]:
weirdos['OBJECT_NAME'].unique()
weirdos[ weirdos['OBJECT_NAME'].str.contains('SES-3')][['idTransponder','mode','band']]

In [ ]:
real.iloc[0]

In [ ]:
real[ real['mode'].str.contains('RX')].iloc[0]

In [ ]:
real.groupby(['band','mode']).count().to_csv('../fcc_band_counts.csv')

In [ ]:
real[ real['band'].str.contains('X')]
# real[ real['eirp'].isnull() ]
real.groupby(['OBJECT_NAME','band'])['id_channels'].count().to_csv('../fcc_by_payload_and_band.csv')